In [1]:
import numpy as np
import pandas as pd
import os
import subprocess
import multiprocessing as mp

# get the path to the root of the repository
root_path = (
    subprocess.check_output(["git", "rev-parse", "--show-toplevel"])
    .decode("utf-8")
    .strip()
)
# set the working directory to the root of the repository
os.chdir(root_path)
# create ./data/09.format_result/ if it doesn't exist
os.makedirs("data/09.format_result", exist_ok=True)